In [27]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import json
import os
import cv2
import pickle
import csv

from PIL import Image
from skimage.feature import hog
from skimage.color import rgb2grey

# Library for scikit-learn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# Library for LIBSVM
from libsvm.svmutil import *
from itertools import combinations
from skimage.data import camera

# Library fror tensorflow and keras
import tensorflow as tf
import tensorflow.keras
import keras
from keras.preprocessing.image import load_img, img_to_array
from keras.applications import imagenet_utils
from keras.models import Model, Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import h5py

In [4]:
# 101 is main dataset, 102 is for testing with smaller dataset
data_dir = "/mnt/c/Users/nhmin/Downloads/food-101/"
bin_n = 16 # Number of bin
project_dir = os.getcwd()
model_dir = "/mnt/c/Users/nhmin/Downloads/"
class_label = {"pad_thai" : 0, "pho" : 1, "ramen" : 2, "spaghetti_bolognese" : 3, "spaghetti_carbonara" : 4}

In [4]:
# Return np img size 227x227
def get_image(path):
    # image resize to 227x227
    img = Image.open(path)
    resized_image = img.resize((227,227), Image.ANTIALIAS)
    return np.array(resized_image)

In [5]:
def load_json(path):
    final_data = dict()
    # Load in json file to create dictionary: key = class label; value = file path
    with open(path, 'r') as file:
        data = json.load(file)
    # Only get information from needed class
    for label in class_label:
        final_data.update({label : data.get(label)})
    return final_data

#=========================
# Exclusive block for showing a sample of how json load data
# data_head = 5
# sample_data = load_json(parent_dir + "/meta/test.json")
# for key, value in sample_data:
#     print(key)
#     print(value[i] for i in range(0,data_head))
#     for i in range(0, data_head):
#         print(value[i])   
#=========================

In [33]:
# def HOG_image(image):
#     np_image = get_image(parent_dir + "images/" + image)
#     # given 32x32 cell
#     image_feature, image_hog = hog(np_image, orientations=8, pixels_per_cell=(8, 8),
#         cells_per_block=(8, 8), block_norm = 'L2-Hys', visualize=True, multichannel=True)
#     return np.array(image_feature)

# def pca_transform(image):
#     ss = StandardScaler()
#     image_ss = ss.fit_transform(image)
#     # Keep 90% of variance
#     pca = PCA(0.85)
#     image_pca = pca.fit_transform(image_ss)
#     return image_pca

def get_key(val):
    for key, value in class_label.items():
        if(val == value): return key
    return None

In [39]:
def create_data(json_data):
    # Create train data
    file_names_list = []
    label_list = []
    for label in json_data.keys():
        file_names = os.listdir(data_dir + '/images/' + label)
        insert_pos = len(file_names)
        for file in file_names:
            file_names_list.append(file)
            label_list.append(class_label.get(label))
    # Create dataframe
    data_df = pd.DataFrame({
        'filename' : file_names_list,
        'label' : label_list
    })
#     # Splitting training and validating data
#     train_df, validate_df = train_test_split(data_df, test_size=0.20, random_state=42)
#     # When splitting, train_df and validate_df contain original index. this is to reset the index
#     train_df = train_df.reset_index(drop=True)
#     validate_df = validate_df.reset_index(drop=True)
    

    return data_df

In [34]:
def create_feature(data_df, pre_model):
    x_tmp = []
    size = len(data_df['filename'])
    for i in range(0,size):
        image = data_df['filename'][i]
        label = data_df['label'][i]
        path = data_dir + 'images/' + get_key(label) + '/' + image
        #image_np = get_image(path)
        image_np = load_img(path, target_size=(227,227))
        image_np = img_to_array(image_np)
        image_np = np.expand_dims(image_np,axis=0)
        image_np = imagenet_utils.preprocess_input(image_np)
        x_tmp.append(image_np)
    x = np.vstack(x_tmp)
    features = pre_model.predict(x, batch_size=32)
    last_layer = pre_model.layers[-1].output_shape
    features_flatten = features.reshape((features.shape[0], last_layer[1] * last_layer[2] * last_layer[3]))
    return features_flatten

In [42]:
# load json file
train_json = load_json(data_dir + '/meta/train.json')
train_df = create_data(train_json)

#model = tf.keras.models.load_model(model_dir + '227-imgsz-32-bsz-0.01-lr-30-ep/saved_model.pb')
model = tf.keras.models.load_model(model_dir + '227-imgsz-32-bsz-0.01-lr-30-ep.ckpt.02-1.64.hdf5')
drop_layer = 12
new_model = tf.keras.Sequential()
for layer in model.layers[:-drop_layer]:
    new_model.add(layer)
new_model.trainable = False
new_model.summary()
# change optimizer when get from flo
# model.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])
train_feature = create_feature(train_df, new_model)
# b = 1 will give prob est
train_model = svm_train(train_df['label'].tolist(), train_feature, '-s 0 -t 0 -c 2.3 -b 1')
svm_save_model('all_food_classification.model', train_model)


Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_54 (Activation)   (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization_18 (Batc (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 25, 25, 256)       221440    
_________________________________________________________________
activation_55 (Activation)   (None, 25, 25, 256)       0         
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 12, 12, 256)     

In [47]:
def test_dataset():
    model = svm_load_model('all_food_classification.model')
    test_json = load_json(data_dir + 'meta/test.json')
    test_df = create_data(test_json)
    test_feature = create_feature(test_df, new_model)
    
    # Predict
    p_label, p_acc, p_val = svm_predict(test_df['label'].tolist(), test_feature, model, '-b 0')
    acc, mse, scc = evaluations(test_df['label'].tolist(), p_label)
    print("Test acc: ", acc)
    # print wront classify image
    counter = 0
    for i in range(0, len(p_label)):
        if(p_label[i] != test_df['label'][i]):
            print(test_df['filename'][i])
            print('True label: ', get_key(test_df['label'][i]), " Predicted label: ", get_key(p_label[i]))
test_dataset()

Model supports probability estimates, but disabled in predicton.
Accuracy = 41.66% (2083/5000) (classification)
Test acc:  41.660000000000004
1004763.jpg
True label:  pad_thai  Predicted label:  ramen
1011059.jpg
True label:  pad_thai  Predicted label:  pho
1051921.jpg
True label:  pad_thai  Predicted label:  ramen
1056648.jpg
True label:  pad_thai  Predicted label:  pho
1102632.jpg
True label:  pad_thai  Predicted label:  ramen
1110311.jpg
True label:  pad_thai  Predicted label:  spaghetti_carbonara
1113102.jpg
True label:  pad_thai  Predicted label:  pho
1113934.jpg
True label:  pad_thai  Predicted label:  pho
1120016.jpg
True label:  pad_thai  Predicted label:  spaghetti_bolognese
1149119.jpg
True label:  pad_thai  Predicted label:  ramen
1157288.jpg
True label:  pad_thai  Predicted label:  pho
116063.jpg
True label:  pad_thai  Predicted label:  ramen
1172295.jpg
True label:  pad_thai  Predicted label:  spaghetti_carbonara
1196664.jpg
True label:  pad_thai  Predicted label:  pho
119

True label:  pad_thai  Predicted label:  pho
53859.jpg
True label:  pad_thai  Predicted label:  pho
539658.jpg
True label:  pad_thai  Predicted label:  ramen
544063.jpg
True label:  pad_thai  Predicted label:  ramen
54549.jpg
True label:  pad_thai  Predicted label:  pho
551989.jpg
True label:  pad_thai  Predicted label:  pho
569063.jpg
True label:  pad_thai  Predicted label:  spaghetti_bolognese
574200.jpg
True label:  pad_thai  Predicted label:  pho
576466.jpg
True label:  pad_thai  Predicted label:  spaghetti_bolognese
579013.jpg
True label:  pad_thai  Predicted label:  spaghetti_carbonara
583272.jpg
True label:  pad_thai  Predicted label:  spaghetti_carbonara
58350.jpg
True label:  pad_thai  Predicted label:  spaghetti_bolognese
585971.jpg
True label:  pad_thai  Predicted label:  spaghetti_bolognese
600929.jpg
True label:  pad_thai  Predicted label:  pho
61986.jpg
True label:  pad_thai  Predicted label:  ramen
640468.jpg
True label:  pad_thai  Predicted label:  pho
653871.jpg
True l

True label:  pho  Predicted label:  spaghetti_bolognese
2866682.jpg
True label:  pho  Predicted label:  pad_thai
2870902.jpg
True label:  pho  Predicted label:  pad_thai
2873419.jpg
True label:  pho  Predicted label:  pad_thai
2877186.jpg
True label:  pho  Predicted label:  pad_thai
2895790.jpg
True label:  pho  Predicted label:  pad_thai
2912658.jpg
True label:  pho  Predicted label:  ramen
2931231.jpg
True label:  pho  Predicted label:  pad_thai
2931384.jpg
True label:  pho  Predicted label:  pad_thai
2932374.jpg
True label:  pho  Predicted label:  ramen
2933981.jpg
True label:  pho  Predicted label:  ramen
2948263.jpg
True label:  pho  Predicted label:  pad_thai
2949163.jpg
True label:  pho  Predicted label:  ramen
2952260.jpg
True label:  pho  Predicted label:  pad_thai
295265.jpg
True label:  pho  Predicted label:  pad_thai
2952717.jpg
True label:  pho  Predicted label:  pad_thai
2954509.jpg
True label:  pho  Predicted label:  pad_thai
2974045.jpg
True label:  pho  Predicted label

846615.jpg
True label:  pho  Predicted label:  pad_thai
873123.jpg
True label:  pho  Predicted label:  pad_thai
874744.jpg
True label:  pho  Predicted label:  spaghetti_carbonara
875789.jpg
True label:  pho  Predicted label:  pad_thai
878749.jpg
True label:  pho  Predicted label:  pad_thai
894781.jpg
True label:  pho  Predicted label:  pad_thai
898135.jpg
True label:  pho  Predicted label:  pad_thai
905481.jpg
True label:  pho  Predicted label:  pad_thai
909041.jpg
True label:  pho  Predicted label:  pad_thai
92244.jpg
True label:  pho  Predicted label:  pad_thai
926.jpg
True label:  pho  Predicted label:  pad_thai
928781.jpg
True label:  pho  Predicted label:  pad_thai
929667.jpg
True label:  pho  Predicted label:  ramen
931906.jpg
True label:  pho  Predicted label:  pad_thai
945282.jpg
True label:  pho  Predicted label:  ramen
95334.jpg
True label:  pho  Predicted label:  ramen
955847.jpg
True label:  pho  Predicted label:  pad_thai
96270.jpg
True label:  pho  Predicted label:  pad_t

True label:  ramen  Predicted label:  pad_thai
245395.jpg
True label:  ramen  Predicted label:  pho
245797.jpg
True label:  ramen  Predicted label:  pho
2475255.jpg
True label:  ramen  Predicted label:  pho
2487409.jpg
True label:  ramen  Predicted label:  pho
2492552.jpg
True label:  ramen  Predicted label:  spaghetti_carbonara
2496198.jpg
True label:  ramen  Predicted label:  pad_thai
2500192.jpg
True label:  ramen  Predicted label:  pho
250129.jpg
True label:  ramen  Predicted label:  spaghetti_bolognese
2503255.jpg
True label:  ramen  Predicted label:  spaghetti_bolognese
2508220.jpg
True label:  ramen  Predicted label:  spaghetti_bolognese
2509239.jpg
True label:  ramen  Predicted label:  spaghetti_carbonara
2511966.jpg
True label:  ramen  Predicted label:  pad_thai
251689.jpg
True label:  ramen  Predicted label:  spaghetti_carbonara
2517585.jpg
True label:  ramen  Predicted label:  pad_thai
2535560.jpg
True label:  ramen  Predicted label:  pho
2538677.jpg
True label:  ramen  Pred

...

True label:  spaghetti_bolognese  Predicted label:  pad_thai
1580734.jpg
True label:  spaghetti_bolognese  Predicted label:  pho
1586675.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
1593292.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
1596512.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
1598961.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
1615859.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
1619227.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
1619474.jpg
True label:  spaghetti_bolognese  Predicted label:  ramen
1622623.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
1624727.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
1625801.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
1635679.jpg
True label:  spaghetti_bolognese  Predicted label:  ramen
1637424.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
1

True label:  spaghetti_bolognese  Predicted label:  pad_thai
2717949.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
2721156.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
2721516.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
2737792.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
274441.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
2756020.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
2759650.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
2759906.jpg
True label:  spaghetti_bolognese  Predicted label:  pho
2760921.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
2762455.jpg
True label:  spaghetti_bolognese  Predicted label:  pho
2766811.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
2773286.jpg
True label:  spaghetti_bolognese  Predicted label:  ramen
2774075.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
2778

True label:  spaghetti_bolognese  Predicted label:  pad_thai
401164.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
418139.jpg
True label:  spaghetti_bolognese  Predicted label:  pho
421748.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
42694.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
447417.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
448640.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
453909.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
460116.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
464467.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
473257.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
473975.jpg
True label:  spaghetti_bolognese  Predicted label:  ramen
475448.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
497303.jpg
True label:  spaghetti_bolognese  Predicted label:  pad_thai
501894.jpg
T

True label:  spaghetti_carbonara  Predicted label:  pad_thai
1844071.jpg
True label:  spaghetti_carbonara  Predicted label:  pad_thai
1846778.jpg
True label:  spaghetti_carbonara  Predicted label:  pad_thai
1853583.jpg
True label:  spaghetti_carbonara  Predicted label:  pad_thai
1853721.jpg
True label:  spaghetti_carbonara  Predicted label:  pad_thai
1853753.jpg
True label:  spaghetti_carbonara  Predicted label:  pho
1857487.jpg
True label:  spaghetti_carbonara  Predicted label:  pad_thai
1857742.jpg
True label:  spaghetti_carbonara  Predicted label:  pad_thai
1861980.jpg
True label:  spaghetti_carbonara  Predicted label:  pho
1862109.jpg
True label:  spaghetti_carbonara  Predicted label:  pad_thai
1891700.jpg
True label:  spaghetti_carbonara  Predicted label:  ramen
1894082.jpg
True label:  spaghetti_carbonara  Predicted label:  pad_thai
1894311.jpg
True label:  spaghetti_carbonara  Predicted label:  pad_thai
1900386.jpg
True label:  spaghetti_carbonara  Predicted label:  pad_thai
190

True label:  spaghetti_carbonara  Predicted label:  pad_thai
3188413.jpg
True label:  spaghetti_carbonara  Predicted label:  pad_thai
3210760.jpg
True label:  spaghetti_carbonara  Predicted label:  pad_thai
3212285.jpg
True label:  spaghetti_carbonara  Predicted label:  pad_thai
3214008.jpg
True label:  spaghetti_carbonara  Predicted label:  pad_thai
3219862.jpg
True label:  spaghetti_carbonara  Predicted label:  pad_thai
3231081.jpg
True label:  spaghetti_carbonara  Predicted label:  ramen
3246555.jpg
True label:  spaghetti_carbonara  Predicted label:  pad_thai
32576.jpg
True label:  spaghetti_carbonara  Predicted label:  pad_thai
3264154.jpg
True label:  spaghetti_carbonara  Predicted label:  pad_thai
3265484.jpg
True label:  spaghetti_carbonara  Predicted label:  pad_thai
3274108.jpg
True label:  spaghetti_carbonara  Predicted label:  ramen
3301599.jpg
True label:  spaghetti_carbonara  Predicted label:  pho
3314539.jpg
True label:  spaghetti_carbonara  Predicted label:  pad_thai
332